# Coherent info

In [1]:
import numpy as np
from collections import Counter
from typing import Tuple, Iterable, Dict, List
from coherentinfo.moebius import MoebiusCode
from coherentinfo.linalg import (finite_field_gauss_jordan_elimination,
                                 finite_field_matrix_rank,
                                 finite_field_inverse,
                                 is_prime)

from numpy.typing import NDArray

from coherentinfo.errormodel import ErrorModel, ErrorModelPoisson
# import galois
import scipy
import timeit

In [2]:
length = 3
width = 3
p = 3
moebius_code = MoebiusCode(length=length, width=width, d=2 * p)
h_z = moebius_code.h_z
h_x = moebius_code.h_x
logical_x = moebius_code.logical_x
logical_z = moebius_code.logical_z

# Vertex syndrome

Now we want to understand how to construct a candidate vector given a certain vertex syndrome. We start with a basic example. Note that the syndrome is mod 2 * p

In [3]:
for _ in range(1000):
    error = np.random.randint(2 * p, size=moebius_code.num_edges)
    syndrome = moebius_code.h_z @ error.T % (2 * p)
    candidate_error = moebius_code.get_vertex_candidate_error(syndrome)
    syndrome_candidate = moebius_code.h_z @ candidate_error.T % (2 * p)
    if np.count_nonzero(syndrome - syndrome_candidate) != 0:
        print("Syndromes do not match")
        break

In [4]:
error = np.random.randint(2 * p, size=moebius_code.num_edges)
candidate_error = moebius_code.get_vertex_candidate_error(syndrome)
error_diff = error - candidate_error
res_com = error_diff @ logical_z.T % (2 * p)
print(res_com == 0 or res_com == p)

True


# Plaquette Syndrome

Now we want to understand how to construct a candidate vector given a certain plaquette syndrome. We start with a basic example. Note that the syndrome is mod 2 * p

In [5]:
for _ in range(100):
    error = np.random.randint(2 * p, size=moebius_code.num_edges)
    syndrome = moebius_code.h_x @ error.T % (2 * p)
    candidate_error = moebius_code.get_plaquette_candidate_error(syndrome)
    syndrome_candidate = moebius_code.h_x @ candidate_error.T % (2 * p)
    if np.count_nonzero(syndrome - syndrome_candidate) != 0:
        print("Syndromes do not match")
        break

error = np.random.randint(2 * p, size=moebius_code.num_edges)
candidate_error = moebius_code.get_plaquette_candidate_error(syndrome)
error_diff = error - candidate_error
print(error_diff @ logical_x.T % (2 * p))

0


# Error model

In [6]:
gamma = 0.01
num_errors = 100
poisson_em = ErrorModelPoisson(num_errors, 2 * p, gamma)

In [7]:
probs = poisson_em.get_probabilities()
print(probs)

[9.80198673e-01 1.96039735e-02 1.96039735e-04 1.30693156e-06
 6.53465782e-09 2.61386313e-11]


In [8]:
my_error = poisson_em.generate_random_error()

In [9]:
sampled_frequencies = np.array([
    np.argwhere(my_error == x).shape[0] / num_errors
    for x in range(2 * p)
])
print(sampled_frequencies)

[1. 0. 0. 0. 0. 0.]


# Compute results conditional entropy

In [14]:
poisson_em_moebius = ErrorModelPoisson(moebius_code.num_edges, 2 * p, gamma)
num_samples = 1000000
result = moebius_code.compute_vertex_conditional_entropy(
    num_samples=num_samples, error_model=poisson_em_moebius)
print(result)

0.019979500507887032


In [15]:
coherent_info = moebius_code.compute_coherent_information(
    num_samples=num_samples, error_model=poisson_em_moebius)
print("Coherent Information: {}".format(coherent_info))

Coherent Information: 0.9801969822450104


In [ ]:
# 2. Use lambda to wrap the method call on the instance
execution_time = timeit.timeit(
    # The lambda function is the callable that timeit executes
    stmt=lambda: moebius_code.compute_plaquette_conditional_entropy(
        num_samples=100000, error_model=poisson_em_moebius),
    number=1  # Run the entire thing 10 times
)

print(f"Time taken for 1 run: {execution_time} seconds")
# Note: The result of the method call itself is ignored by timeit.timeit,
# which only records the execution time.

Time taken for 10 runs: 16.402176355000847 seconds


In [ ]:
result_probs = moebius_code.compute_plaquette_syndrome_chi_probabilities(
    100000, poisson_em_moebius)

In [44]:
print(result_probs)

{'0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0': [0.40737000000026663, 0.0], '0_0_0_0_5_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_1_0': [0.008379999999999827, 0.0], '0_0_0_0_0_0_0_1_0_0_0_0_0_5_0_0_0_0_0_0_0_0_0_0_0': [0.0002, 0.0], '0_0_0_0_0_5_0_0_0_0_1_0_0_5_1_1_0_0_1_0_0_0_0_0_0': [0.0, 1e-05], '0_0_0_0_0_1_0_0_0_1_0_0_0_1_5_0_0_0_0_0_0_0_5_1_0': [0.0, 1e-05], '0_0_0_0_0_0_0_0_0_0_5_1_1_0_0_0_0_5_0_0_0_0_0_0_0': [0.00016, 0.0], '0_0_0_0_0_0_0_0_0_1_0_0_0_0_0_0_0_0_5_0_0_0_0_0_0': [0.007779999999999851, 0.0], '0_0_0_0_0_5_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0': [1e-05, 0.0], '0_0_0_0_1_0_0_0_0_5_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0': [0.008409999999999826, 0.0], '0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_5_0_0_0_5': [0.0, 0.0077999999999998505], '0_0_0_0_0_1_0_0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0': [0.0, 0.008379999999999827], '0_0_0_5_0_0_0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0': [0.007989999999999843, 0.0], '0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_5_1_0_0_0_0_0_0_0': [0.008169999999999835, 0.0], '0_0_

In [41]:
key_list = list(result_probs.keys())

In [42]:
key_list

['0_5_5_1_5_5_1_0_1_1_1_0_0_5_0_0_0_5_0_0_0_0_0_1_0',
 '3_2_5_2_5_1_1_2_4_1_5_0_5_0_1_1_1_5_5_5_0_0_2_0_0',
 '5_5_1_5_5_0_1_5_1_0_5_1_0_0_0_5_0_0_1_0_0_1_0_5_0',
 '0_1_5_0_0_0_0_5_1_0_5_1_1_0_1_0_0_0_0_4_5_0_0_5_0',
 '4_1_5_5_3_0_0_5_1_5_4_2_0_4_0_0_0_2_0_0_5_1_0_1_0',
 '0_0_0_0_0_4_0_0_0_0_2_0_1_1_5_5_0_4_0_5_4_1_1_0_5',
 '5_0_5_1_5_0_1_0_1_0_0_5_0_5_0_1_1_0_1_1_5_0_5_0_0',
 '0_0_5_1_0_0_0_0_2_0_2_0_0_4_1_4_1_0_0_5_5_0_5_1_0',
 '0_0_0_5_0_1_0_1_0_1_0_0_0_5_0_0_0_0_2_0_0_1_5_5_0',
 '0_0_0_0_5_1_5_0_0_1_0_0_0_0_0_1_0_0_5_0_0_0_0_1_1']

In [ ]:
entropy = 0.0
key = key_list[1]
prob_syndrome = sum(result_probs[key])
cond_prob_syndrome_chi_zero = result_probs[key][0] / prob_syndrome
cond_prob_syndrome_chi_one = result_probs[key][1] / prob_syndrome
entropy += -scipy.special.xlogy(result_probs[key][0],
                                cond_prob_syndrome_chi_zero)
entropy += -scipy.special.xlogy(result_probs[key][1],
                                cond_prob_syndrome_chi_one)
print(entropy)

0.0


In [ ]:
scipy.special.xlogy(result_probs[key][0], cond_prob_syndrome_chi_zero)

np.float64(0.0)

In [ ]:
scipy.special.xlogy(result_probs[key][1], cond_prob_syndrome_chi_one)

np.float64(0.0)

In [32]:
print(result_probs[key][0])
print(cond_prob_syndrome_chi_zero)

0.0
0.0


In [33]:
print(result_probs[key][1])
print(cond_prob_syndrome_chi_one)

0.1
1.0


In [ ]:
import numpy as np

# Use the natural logarithm of 2 for the base change
LOG_2 = np.log(2)


def x_log2_x(x):
    """
    Computes x * log2(x), safely handling x=0 by returning 0.
    """
    # xlogy(x, y) computes x * log(y) where log is the natural log (ln)
    # The division by LOG_2 converts the base from e to 2:
    # x * ln(x) / ln(2) = x * log2(x)
    return np.xlogy(x, x) / LOG_2

# --- Example Usage ---


# Case 1: x = 0
x1 = 0.0
result1 = x_log2_x(x1)
print(f"x={x1}: {result1}")
# Output will be: x=0.0: 0.0

AttributeError: module 'numpy' has no attribute 'xlogy'

In [16]:
pip list

Package                 Version     Editable project location
----------------------- ----------- -----------------------------------------------------
asttokens               3.0.0
coherentinfo            0.0.1       /home/alessandrociani/git/github/Coherent-Information
comm                    0.2.3
contourpy               1.3.3
cycler                  0.12.1
debugpy                 1.8.17
decorator               5.2.1
executing               2.2.1
fonttools               4.60.1
galois                  0.4.7
iniconfig               2.3.0
ipykernel               7.0.1
ipython                 9.6.0
ipython_pygments_lexers 1.1.1
jedi                    0.19.2
jupyter_client          8.6.3
jupyter_core            5.9.1
kiwisolver              1.4.9
llvmlite                0.45.1
matplotlib              3.10.7
matplotlib-inline       0.1.7
nest-asyncio            1.6.0
numba                   0.62.1
numpy                   2.3.4
packaging               25.0
parso                   0.8.5
pe

In [17]:
scipy.special.xlogy(np.exp(1), np.exp(1))

np.float64(2.718281828459045)

In [20]:
np.log(np.exp(1))

np.float64(1.0)